In [14]:
!pip install duckdb

In [15]:
import duckdb

In [16]:
duckdb.query("SELECT * FROM read_csv_auto('/content/cab_ride_data.csv') LIMIT 5").df()

,ride_id,date,pickup_location,dropoff_location,fare,tip,total_amount,total_distance_km,duration_min,payment_type
0,1,2024-02-08,Airport,Downtown,24.10,2.89,26.99,29.62,91.26,Cash
1,2,2024-02-21,Business District,Airport,8.92,1.94,10.86,8.87,33.83,Digital Wallet
2,3,2024-01-29,Suburbs,Business District,41.71,3.51,45.22,3.86,12.21,Credit Card
3,4,2024-01-15,Business District,Airport,21.81,4.50,26.31,25.05,114.99,Credit Card
4,5,2024-02-12,Suburbs,Airport,28.69,0.21,28.90,21.49,62.89,Cash


In [17]:
import pandas as pd
from scipy.stats import chi2_contingency

# Query data
query = """
    SELECT
        payment_type,
        SUM(CASE WHEN tip > 0 THEN 1 ELSE 0 END) AS tipped,
        SUM(CASE WHEN tip = 0 THEN 1 ELSE 0 END) AS no_tip
    FROM read_csv('/content/cab_ride_data.csv')
    GROUP BY payment_type;
"""
df = duckdb.query(query).df()

# Create contingency table
contingency_table = df[['tipped', 'no_tip']].values

# Perform chi-square test
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

# Display results
print(f"Chi-Square Statistic: {chi2_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

Chi-Square Statistic: 1.8509
P-Value: 0.3964


In [18]:
duckdb.query("""
    SELECT corr(duration_min, tip_percentage) AS correlation
    FROM (
        SELECT duration_min,
               (tip / fare) * 100 AS tip_percentage
        FROM read_csv('/content/cab_ride_data.csv')
        WHERE tip IS NOT NULL AND fare > 0
    )
""").df()


,correlation
0,-0.016022
